# Proyecto Integrador

## Segundo avance: Conexión con la API Yelp

##### Importamos las librerías necesarias:

In [1]:
import pandas as pd
import requests

### 1. Configuración de la API

In [2]:
# url de la API de Yelp
api_url = "https://api.yelp.com/v3/businesses/search"

# API Key de Yelp
api_key = 'api_key_valida_aqui'

# Configuración de los headers para la autenticación
headers = {'Authorization': f'Bearer {api_key}'}

Mi ciudad

In [3]:
ciudad = 'San Diego'

### 2. Primera solicitud a Yelp

In [4]:
# construimos los parámetros de la consulta usando un diccionario: 'term', 'localización' y 'límite'
params = {'term': 'restaurants', 'location': ciudad, 'limit': 50}

# hacemos la solicitud GET a la API de Yelp y la llamamos 'response'
response = requests.get(api_url, headers=headers, params=params)

# convertimos la respuesta JSON en un diccionario de Python
data = response.json()

# imprimimos los datos obtenidos
data

{'error': {'code': 'VALIDATION_ERROR',
  'description': "'Bearer api_key_valida_aqui' does not match '^(?i)Bearer [A-Za-z0-9\\\\-\\\\_]{128}$'",
  'field': 'Authorization',
  'instance': 'Bearer api_key_valida_aqui'}}

In [5]:
type(data)

dict

In [6]:
data.keys()

dict_keys(['error'])

In [7]:
data['total']

KeyError: 'total'

In [ ]:
data['region']

{'center': {'longitude': -117.15408325195312, 'latitude': 32.790569394537286}}

In [ ]:
data['businesses']

[{'id': 'Sa_qcnc7ZgzSOylf3plTRA',
  'alias': 'c-level-san-diego-6',
  'name': 'C Level',
  'image_url': 'https://s3-media0.fl.yelpcdn.com/bphoto/rE2Ti2kPJTW_xv2FkW-8Lg/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/c-level-san-diego-6?adjust_creative=GWOCZh9-BmZxtdsAjr7Gug&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=GWOCZh9-BmZxtdsAjr7Gug',
  'review_count': 5329,
  'categories': [{'alias': 'newamerican', 'title': 'New American'},
   {'alias': 'seafood', 'title': 'Seafood'},
   {'alias': 'steak', 'title': 'Steakhouses'}],
  'rating': 4.3,
  'coordinates': {'latitude': 32.72412, 'longitude': -117.18857},
  'transactions': [],
  'price': '$$',
  'location': {'address1': '880 Harbor Island Dr',
   'address2': '',
   'address3': '',
   'city': 'San Diego',
   'zip_code': '92101',
   'country': 'US',
   'state': 'CA',
   'display_address': ['880 Harbor Island Dr', 'San Diego, CA 92101']},
  'phone': '+16192986802',
  'display_phone': '(619) 298-68

In [ ]:
len(data['businesses'])

50

In [ ]:
data['businesses'][17] # Información del restaurante en la posición 18va (comienza la cuenta en cero)

{'id': '0gi0ArxUwbz5wNqisA0Q6w',
 'alias': 'leila-san-diego-3',
 'name': 'Leila',
 'image_url': 'https://s3-media0.fl.yelpcdn.com/bphoto/8ObW2RlGABKXHBnHE-eKqQ/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/leila-san-diego-3?adjust_creative=GWOCZh9-BmZxtdsAjr7Gug&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=GWOCZh9-BmZxtdsAjr7Gug',
 'review_count': 907,
 'categories': [{'alias': 'mideastern', 'title': 'Middle Eastern'},
  {'alias': 'cocktailbars', 'title': 'Cocktail Bars'},
  {'alias': 'halal', 'title': 'Halal'}],
 'rating': 4.0,
 'coordinates': {'latitude': 32.7495329005057,
  'longitude': -117.13031122021752},
 'transactions': [],
 'price': '$$$$',
 'location': {'address1': '3956 30th St',
  'address2': None,
  'address3': '',
  'city': 'San Diego',
  'zip_code': '92104',
  'country': 'US',
  'state': 'CA',
  'display_address': ['3956 30th St', 'San Diego, CA 92104']},
 'phone': '+18587233766',
 'display_phone': '(858) 723-3766',
 'distance': 

¿Cómo extraer la mayor cantidad de resultados posibles desde la API y convertir la respuesta en un DataFrame?

###### La API cuenta con información relativa a 4000 restaurantes, pero en cada consulta se extrae una página con 50 resultados. Por lo tanto, para extraer todos los datos será necesario iterar el proceso. El parámetro offset de la API indica el número de registros a avanzar en cada consulta (avanzamos de 50 en 50).

### 3. Paginación y normalización

In [ ]:
# iterar y fijamos el offset
offset = 0

# ciudad
ciudad = 'San Diego'

# creo una lista que contenga los dataframes creados en cada consulta
all_restaurants = [] # [dataframe1, dataframe2, dataframe3, dataframe4]

# voy a iterar hasta alcanzar 200 resultados (límite de Yelp)
while offset <= 150: # actualizo el parámetro offset en cada iteración
    # parámetros de la API
    params = {'term': 'restaurants', 'location': ciudad, 'limit': 50, 'offset': offset}

    # creo una variable 'response que albergue el resultado de la consulta a la API de Yelp
    response = requests.get(api_url, headers=headers, params=params)

    # convierto la respuesta en un diccionario de Python
    data = response.json()

      # voy a usar el método json_normalize de pandas para convertir la lista de negocios en un DataFrame
    rest = pd.json_normalize(data['businesses'], sep='_', record_path=['categories'],
                             meta=['name', 'rating', 'review_count', 'is_closed', 'price', ['location.address1']],
                             errors='ignore')
    
    rest['city'] = ciudad
    
    # agrego el dataframe a la lista
    all_restaurants.append(rest)

    # incremento el offset en 50 para la siguiente iteración
    offset = offset + 50

    # condición de salida del bucle en caso de no haber más negocios
    if 'businesses' not in data or len(data['businesses']) == 0:
       break

In [ ]:
type(rest)

pandas.core.frame.DataFrame

In [ ]:
len(rest)

110

In [ ]:
len(all_restaurants)

4

In [ ]:
rest = pd.concat(all_restaurants, ignore_index=True)
rest

,alias,title,name,rating,review_count,is_closed,price,location.address1,city
0,newamerican,New American,C Level,4.3,5329,False,$$,NaN,San Diego
1,seafood,Seafood,C Level,4.3,5329,False,$$,NaN,San Diego
2,steak,Steakhouses,C Level,4.3,5329,False,$$,NaN,San Diego
3,italian,Italian,Ristorante Illando,4.4,3329,False,$$,NaN,San Diego
4,latin,Latin American,Flama Llama,4.7,3833,False,$$,NaN,San Diego
...,...,...,...,...,...,...,...,...,...
484,cocktailbars,Cocktail Bars,EE NAMI Tonkatsu Izakaya,4.5,659,False,$$,NaN,San Diego
485,thai,Thai,Tid Din,4.4,8,False,NaN,NaN,San Diego
486,lounges,Lounges,Harbor & Sky Rooftop Bar and Lounge,4.0,126,False,$$,NaN,San Diego
487,newamerican,New American,Harbor & Sky Rooftop Bar and Lounge,4.0,126,False,$$,NaN,San Diego


### 4. Agrupación de categorías

In [ ]:
categoria_por_restaurante = (rest.groupby('name')['title']
    .apply(lambda x: ", ".join(x.unique()))
    .reset_index()
    .rename(columns={'title':'categories'}))

##### Obtengo el DF base sin duplicaciones:

In [ ]:
cols_base = ['name', 'rating', 'review_count', 'is_closed',
    'price', 'city']

rest_base = rest[cols_base].drop_duplicates(subset='name')


##### Uno las categorías con info base

In [ ]:
df_final = rest_base.merge(categoria_por_restaurante, on='name', how='left')
df_final.head()

,name,rating,review_count,is_closed,price,city,categories
0,C Level,4.3,5329,False,$$,San Diego,"New American, Seafood, Steakhouses"
1,Ristorante Illando,4.4,3329,False,$$,San Diego,Italian
2,Flama Llama,4.7,3833,False,$$,San Diego,"Latin American, Peruvian, Asian Fusion"
3,Cesarina,4.7,3297,False,$$$,San Diego,"Italian, Cocktail Bars, Desserts"
4,The Remy,4.5,533,False,NaN,San Diego,"Steakhouses, Desserts, Salad"


### 5. Verifico la calidad de los datos

##### Valores nulos:

In [ ]:
df_final.isnull().sum()

name             0
rating           0
review_count     0
is_closed        0
price           38
city             0
categories       0
dtype: int64

##### Duplicados:

In [ ]:
df_final['name'].duplicated().sum()

np.int64(0)

In [ ]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197 entries, 0 to 196
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          197 non-null    object
 1   rating        197 non-null    object
 2   review_count  197 non-null    object
 3   is_closed     197 non-null    object
 4   price         159 non-null    object
 5   city          197 non-null    object
 6   categories    197 non-null    object
dtypes: object(7)
memory usage: 10.9+ KB


### 6. Tratamiento de los datos nulos

##### Decido darle un tratamiento a los valores nulos que me trae la columna 'price'. En particular, colocarle la leyenda 'Unknown', pero en una nueva columna llamada 'price_clean':

In [ ]:
df_final['price_clean'] = df_final['price'].fillna('Unknown')

##### Verfico el tratamiento de estas celdas trayendo las primeras 5 o hasta ver un valor NaN junto a un valor Unknown:

In [ ]:
df_final[['price', 'price_clean']].head(5)

,price,price_clean
0,$$,$$
1,$$,$$
2,$$,$$
3,$$$,$$$
4,NaN,Unknown


###### Arriba se observa que en el registro con valores 'NaN', en la celda price_clean se lee 'Unknown' al tiempo que en la misma columna (clean) se respeta el valor del registro original.

In [ ]:
df_final.head(10)

,name,rating,review_count,is_closed,price,city,categories,price_clean
0,C Level,4.3,5329,False,$$,San Diego,"New American, Seafood, Steakhouses",$$
1,Ristorante Illando,4.4,3329,False,$$,San Diego,Italian,$$
2,Flama Llama,4.7,3833,False,$$,San Diego,"Latin American, Peruvian, Asian Fusion",$$
3,Cesarina,4.7,3297,False,$$$,San Diego,"Italian, Cocktail Bars, Desserts",$$$
4,The Remy,4.5,533,False,NaN,San Diego,"Steakhouses, Desserts, Salad",Unknown
5,Allegro Restaurant and Bar,4.0,2049,False,$$$,San Diego,"Italian, Venues & Event Spaces, Seafood",$$$
6,Wolf in the Woods,4.6,476,False,$$$,San Diego,"Wine Bars, Modern European, Tapas Bars",$$$
7,formoosa,4.7,5212,False,$$,San Diego,"Taiwanese, Coffee & Tea, Noodles",$$
8,Steamy Piggy,4.4,7510,False,$$,San Diego,"Chinese, Asian Fusion, Ramen",$$
9,Gravity Heights - Mission Valley,4.6,448,False,$$,San Diego,"Breakfast & Brunch, American, Brewpubs",$$


### 7. Exporto a CSV

In [ ]:
df_final.to_csv("yelp_restaurants_sandiego.csv", index=False)